In [1]:
import sys
sys.path.append('../')

from Globals import MULTI_VISIT_PKL, UNIQUE_ATC_CSV, UNIQUE_ICD_CSV
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from torch_geometric.nn import Sequential, GCNConv
 

C:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
GLOBAL_DATA_PATH = "../Data"
ATC4 = "ATC4"
ICD9_CODE = "ICD9_CODE"

In [3]:
records = pd.read_pickle(os.path.join(GLOBAL_DATA_PATH, MULTI_VISIT_PKL))

# Retrieve unique medication codes and diagnosis codes
med_vocab = pd.read_csv(os.path.join(GLOBAL_DATA_PATH, UNIQUE_ATC_CSV))[ATC4]
diag_vocab = pd.read_csv(os.path.join(GLOBAL_DATA_PATH, UNIQUE_ICD_CSV))[ICD9_CODE]

med_vocab = med_vocab.values.tolist()
diag_vocab = diag_vocab.values.tolist()
#print(med_vocab.values.tolist())

print(records.head(5))
print(records['ATC4'].head(5))

   SUBJECT_ID  HADM_ID                                               ATC4  \
0           4   185777  [A12BA, B05XA, N02BE, A10AE, R05DB, R05DA, R05...   
1           6   107064  [A12AA, D11AX, C07AB, C08CA, D11AH, L04AD, C03...   
2           9   150750  [A12BA, B05XA, J01MA, S01AE, C07AB, C02DB, A01...   
3          11   194540  [B01AB, C05BA, S01XA, A06AA, A06AB, N02BE, N02...   
4          12   112213  [N05BA, A12AA, D11AX, B01AB, C05BA, S01XA, C07...   

                                           ICD9_CODE  
0  [042, 1363, 7994, 2763, 7907, 5715, 04111, V09...  
1  [40391, 4440, 9972, 2766, 2767, 2859, 2753, V1...  
2                [431, 5070, 4280, 5849, 2765, 4019]  
3                                             [1913]  
4  [1570, 57410, 9971, 4275, 99811, 4019, 5680, 5...  
0    [A12BA, B05XA, N02BE, A10AE, R05DB, R05DA, R05...
1    [A12AA, D11AX, C07AB, C08CA, D11AH, L04AD, C03...
2    [A12BA, B05XA, J01MA, S01AE, C07AB, C02DB, A01...
3    [B01AB, C05BA, S01XA, A06AA, A06AB, N

In [23]:
import dill
import pickle as pkl

gamenet_data_path = '../GAMENet_Data'
voc_file = 'voc_final.pkl'
file_obj = open(os.path.join(gamenet_data_path, voc_file), 'rb')
med_voc = pkl.load(file_obj)
med_voc_size = len(med_voc.idx2word)

ehr_adj = np.zeros((med_voc_size, med_voc_size))
for patient in records:
    for adm in patient:
        med_set = adm[2]
        for i, med_i in enumerate(med_set):
            for j, med_j in enumerate(med_set):
                if j<=i:
                    continue
                ehr_adj[med_i, med_j] = 1
                ehr_adj[med_j, med_i] = 1
print(ehr_adj)
#dill.dump(ehr_adj, open('ehr_adj_final.pkl', 'wb'))  

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 664: character maps to <undefined>

In [4]:
# Create adjacency matrix from medication codes
N = len(med_vocab)

adj_matrix = torch.zeros(N, N)

for patient in records[ATC4]:
    for visit in patient:
        for i, code_i in enumerate(visit):
            for j, code_j in enumerate(visit):
                if j > i:
                    adj_matrix[code_i, code_j] = 1
                    adj_matrix[code_j, code_i] = 1

print(adj_matrix)      

TypeError: new(): invalid data type 'str'

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, emb_dim=16):
        super(RNN, self).__init__()

        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=emb_dim)
        self.rnn = nn.GRU(input_size=emb_dim, hidden_size=emb_dim, batch_first=True)
        self.init_weights()
    
    def forward(self, codes):        
        emb_list = []
        for code in codes:
            emb = None
            emb = self.embeddings(torch.tensor(code))
            
            # take the mean and make one sample per batch
            emb_mean = emb.mean(dim=0).unsqueeze(dim=0)
            emb_list.append(emb_mean)
            
        emb_seq = torch.cat(emb_list, dim=0).unsqueeze(dim=0)
        
        result, _ = self.rnn(emb_seq)
        
        return result
    
    def init_weights(self):
        torch.nn.init.normal_(self.embeddings.weight)
        for param in self.rnn.parameters():
            if len(param.shape) >= 2:
                torch.nn.init.orthogonal_(param.data)
            else:
                torch.nn.init.normal_(param.data)
    

In [ ]:
class PatientQuery(nn.Module):
    def __init__(self, diag_vocab, emb_dim=16):
        super(PatientQuery, self).__init__()
        self.diag_rnn = RNN(diag_vocab, emb_dim)
        self.linear = nn.Linear(in_features=emb_dim * 2, out_features=emb_dim, bias=True)
        
    def forward(self, codes_diag,):
        """
        Input:
            codes_diag: [[diag codes for visit1], [diag codes for visit2], ...]
                - e.g., [[0,1], [1,2]]  
            codes_prod: [[prod codes for visit1], [prod codes for visit2], ...]
                - e.g., [[0,1], [1,2]]  
        output:
            query embedding:
                - size: (#visits, emd_dim)
        """
        
        """
        TODO: get diag and prod embedding by self.rnn_diag, self.rnn_prod
        """
        diag_emb = self.diag_rnn(codes_diag)
        
        """
        TODO: concat emb_diag and emb_prod, then tranfrom by self.linear
        """
        emb_cat = None
        result = None
        
        #emb_cat = torch.cat((emb_diag, emb_prod), 2)
        #result = self.linear(emb_cat)
        #result = result.squeeze(0)
        #print(result.size())
        #return result

        return diag_emb

In [ ]:
class GCN(nn.Module):
    def __init__(self, vocab_med, adj, emb_dim=16):
        super(GCN, self).__init__()
        adj = self.normalize(adj + np.eye(adj.shape[0]))
        self.adj = torch.FloatTensor(adj)
        
        self.gcn1 = GCNConv(in_channels=vocab_med, out_channels=emb_dim)
        self.gcn2 = GCNConv(in_channels=emb_dim, out_channels=emb_dim)
        
        # self.gcn1 = GraphConvolution(in_features=vocab_med, out_features=emb_dim)
        # self.gcn2 = GraphConvolution(in_features=emb_dim, out_features=emb_dim)
        
        # the initial feature
        self.x = torch.eye(vocab_med)
    
    def forward(self):
        """
        TODO: 
            1 (already done). use self.gcn1 for the first graph convolution
                - remember to use self.adj
            2. use F.relu() as the activation function
            3. use self.gcn2
        """
        x = self.gcn1(self.x, self.adj)
        x = F.relu(x)
        x = self.gcn2(x, self.adj)
        
        return x
    
    def normalize(self, adj):
        adj = adj / (adj @ np.ones(adj.shape) + 1e-8)
        return adj

In [ ]:
class MemoryBank(nn.Module):
    def __init__(self, vocab_med, adj_ehr, adj_ddi, emb_dim=16):
        super(MemoryBank, self).__init__()
        """
        FUNCTIONALITY: conbime information from EHR graph and DDI graph
        """
        
        """
        TODO:
            1. use GCN for EHR graph
                - using adj_ehr
                - using emb_dim as the ouput dimension
            2. use GCN for DDI graph
                - using adj_ddi
                - using emb_dim as the ouput dimension
            3 (already done). design a learnable weight between them
        """
        
        self.gcn_ehr = GCN(vocab_med=vocab_med, adj=adj_ehr, emb_dim=emb_dim)
        
        self.weight = nn.Parameter(torch.FloatTensor(1))
        self.weight.data.uniform_(-0.1, 0.1)
        
    def forward(self):
        """
        TODO: 
            1. get ehr graph feature
                - using .forward() function
            2. get ddi graph feature
                - using .forward() function
            3 (already done). get weighted information
        """
        
        info_ehr = self.gcn_ehr.forward()
        
        info_comb = info_ehr * self.weight
        
        return info_comb

In [ ]:
class DynamicMemory(nn.Module):
    def __init__(self, vocab_med):
        super(DynamicMemory, self).__init__()
        """
        FUNCTIONALITY: generate a historical mapping: query embedding -> multi-hot medication vector
        """
        self.vocab_med = vocab_med
    
    def forward(self, queries, codes_med):
        """
        Input:  queries
                    - this is the historical query embedding, given by PatientQuery Module
                    - size: (#visits - 1, emb_dim), delete the current query
                codes_med
                    - this is the historical groud truth med vector
                    - format: a list of length (#visits - 1)
        """
        
        DM_key = queries
        DM_value = np.zeros((queries.shape[0], self.vocab_med))
        
        """
        TODO: transform codes_med to multi-hot and filling DM_value row by row (visit by visit)
        """
        #print(DM_key)
        #print(codes_med)
        #print(DM_value)
        for i in range(len(codes_med)):
            for j in codes_med[i]:
                DM_value[i][j] = 1
        
        # use torch.FloatTensor for DM_value
        DM_value = torch.FloatTensor(DM_value)
        
        return DM_key, DM_value

In [ ]:
class Fact1(nn.Module):
    def __init__(self, queries):
        super(Fact1, self).__init__()
        """
        FUNCTIONALITY: extract the final embedding from input queries, q^t
        """
        self.queries = queries
    
    def forward(self):
        """
        TODO: assign the last embedding to result
                - final size: (1, emb_dim)
                - make sure the output size is **not** a vector format (emb_dim,)
        """
        result = None
        #print("from fact1, queries size")
        #print(self.queries.size())
        result = self.queries[-1]
        
        #print("from fact1, result size")
        #print(result.size())
        result = result.unsqueeze(0)
        #print(result.size())
        return result
    

class Fact2(nn.Module):
    def __init__(self, query, MB):
        super(Fact2, self).__init__()
        """
        FUNCTIONALITY: get attention information from MB, o^t_b
        Input:
            query
                - this is the last embedding
            MB
                - is the memory bank
        """
        self.query = query
        self.MB = MB
    
    def forward(self):
        """
        TODO: 
            1. get the attention weight between query and each row of MB, adding to attn_score
                - use torch.mm()
                - tip: need to transpose MB, using MB.t()
            2. use F.softmax(, dim=1) to compute the attention matrix, attn_matrix
            3. get the final result from attn_matrix and MB
                - use torch.mm(,)
        """
        attn_score = None
        attn_matrix = None
        result = None
        #print("query")
        #print(self.query.size())
        #print("mb")
        #print(self.MB.size())
        attn_score = torch.mm(self.query, self.MB.t())
        attn_matrix = F.softmax(attn_score, dim=1)
        result = torch.mm(attn_matrix, self.MB)
        
        return result
    
class Fact3(nn.Module):
    def __init__(self, query, MB, DM_key, DM_value):
        super(Fact3, self).__init__()
        """
        FUNCTIONALITY: similar to Fact2, get information from the DM, o_d^t
        """
        self.query = query
        self.MB = MB
        self.DM_key = DM_key
        self.DM_value = DM_value
    
    def forward(self):
        attn = F.softmax(torch.mm(self.query, self.DM_key.t()), dim=1)
        value = torch.mm(attn, self.DM_value)
        out = torch.mm(value, self.MB)
        return out

In [ ]:
class OutNet(nn.Module):
    def __init__(self, vocab_med, emb_dim=16):
        super(OutNet, self).__init__()
        """
        FUNCTIONALITY: combine fact1, fact2, fact3 to do final prediction
        """
        
        """
        TODO:
            1. build the first linear layer
                - input size: 3 * emb_dim
                - end size: 2 * emb_dim
                - use bias=True option
            2. build the second linear layer
                - input size: 2 * emb_dim
                - end size: vocab_med
                - use bias=True option
        """
        self.fc1 = None
        self.fc2 = None
        
        self.fc1 = nn.Linear(in_features=3 * emb_dim, out_features=2 * emb_dim, bias=True)
        self.fc2 = nn.Linear(in_features=2 * emb_dim, out_features=vocab_med, bias=True)
        
    def forward(self, fact1, fact2, fact3):
        """
        Input:
            fact1, fact2, fact3:
                - three facts q^t, o^t_b, o^t_d
                - each size: (1, emb_dim)
        """
        
        """
        TODO: 
            1. concat fact1, fact2, fact3, and assign to memory_out
                - size: (1, 3 * emb_dim)
            2. pass through the first linear layer
            3. use F.relu() as the activation
            3. pass through the second linear layer
        """
        memory_out = None
        result = None
        
        memory_out = torch.cat((fact1, fact2, fact3), dim=1)
        #print(memory_out.size())
        
        result = self.fc1(memory_out)
        result = F.relu(result)
        result = self.fc2(result)
        
        return result

In [ ]:
class GAMENet(nn.Module):
    def __init__(self, vocab_diag, vocab_prod, vocab_med, adj_ehr, adj_ddi, emb_dim=128):
        super(GAMENet, self).__init__()
        """
        FUNCTIONALITY: integrate the whole model together
        """
        
        """
        TODO: 
            1. build a patient query network
                - PatientQuery
                - use vocab_diag, vocab_prod, emb_dim
            2. build a memory bank
                - MemoryBank
                - use vocab_med, adj_ehr, adj_ddi, emb_dim
            3. build a dynamic memory
                - DynamicMemory
                - use vocab_med
            4. build a output network
                - OutNet
                - vocab_med, emb_dim
        """
        self.patient = None
        self.memorybank = None
        self.dynamicmemory = None
        self.outnet = None
        
        self.patient = PatientQuery(vocab_diag, vocab_prod, emb_dim)
        self.memorybank = MemoryBank(vocab_med, adj_ehr, adj_ddi, emb_dim)
        self.dynamicmemory = DynamicMemory(vocab_med)
        self.outnet = OutNet(vocab_med, emb_dim)
        
    def forward(self, codes_diag, codes_prod, codes_med):
        """
        input:
            codes_diag
                - a list of length #visits
                - each element is also itself a list
            codes_prod
                - a list of length #visits
                - each element is also itself a list
            codes_med
                - a list of length #visits - 1
                - each element is also itself a list
        """
        
        # get patient query embedding (#visit, emb_dim*2)
        queries = self.patient(codes_diag, codes_prod)
        """
        TODO:
            1. build a memory bank, assign to MB
                - use self.memorybank
            2. build a dynamic memory, assign to DM_key and DM_value
                - use self.dynamicmemory
                - use queries and codes_med as features
        """
        MB = None
        DM_key, DM_value = None, None
           
        MB = self.memorybank()
        DM_key, DM_value = self.dynamicmemory(queries, codes_med)
        #print("queries")
        #print(queries.size())
        # extract three memory outputs, assign to fact1, fact2, fact3
        fact1 = Fact1(queries)()
        #print(MB.size())
        fact2 = Fact2(fact1, MB)()
        fact3 = Fact3(fact1, MB, DM_key, DM_value)()

        # get the final output
        result = self.outnet(fact1, fact2, fact3)
        
        return result